In [1]:
import pandas as pd
from pathlib import Path
import pickle
import glob
from tqdm.auto import tqdm
import json


/Users/smukherjee/.pyenv/versions/3.10.14/envs/openml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
eval_path = Path("../../data/evaluation/")

In [3]:
load_eval_queries = pd.read_csv(eval_path / "LLM Evaluation - Topic Queries.csv")[["Topic", "Dataset IDs"]]

In [4]:
# read templates and add information to the queries
with open(eval_path/ "query_templates.txt", "r") as f:
    query_templates = f.readlines()
    query_templates = [x.strip() for x in query_templates]

In [5]:
query_key_dict = {}
for template in query_templates:
    for row in load_eval_queries.itertuples():
        new_query = f"{template} {row[1]}"
        # load_eval_queries.at[query, "Query"] = new_query
        if new_query not in query_key_dict:
            query_key_dict[new_query] = row[2]

In [6]:
json.dump(query_key_dict, open(eval_path / "query_key_dict.json", "w"))

In [7]:
query_key_dict = json.load(open(eval_path / "query_key_dict.json", "r"))

In [8]:

def create_results_dict(csv_files, df_queries):
    # Create a dictionary to store the results
    results_dict = {}
    
    # Convert the df_queries DataFrame to a dictionary for faster lookup
    query_dict = dict(zip(df_queries["query"], df_queries["label"]))

    for exp_path in tqdm(csv_files):
        folder_name = Path(exp_path).parent.name
        exp = pd.read_csv(exp_path)

        # Create y_pred
        exp["y_pred"] = exp["did"].astype(str)

        # Map true labels using the query dictionary
        exp["y_true"] = exp["query"].map(query_dict)

        # Calculate the number of correct and wrong predictions
        exp["correct"] = exp.apply(lambda row: row["y_pred"] in row["y_true"], axis=1)
        correct = exp["correct"].sum()
        wrong = len(exp) - correct

        # Store results in the dictionary
        results_dict[folder_name] = {"correct": correct, "wrong": wrong}
    
    return results_dict

In [9]:

# glob all csv files in the experiments directory
csv_files = glob.glob(str(eval_path / "*/*/results.csv"))
csv_files

['../../data/evaluation/BAAI_bge-large-en-v1.5/BAAI_bge-large-en-v1.5_qwen2_1.5b/results.csv']

In [10]:
exp_path = csv_files[0]

In [11]:
folder_name = Path(exp_path).parent.name
exp = pd.read_csv(exp_path).rename(columns={"did": "y_pred"})
exp.head()

,Unnamed: 0,y_pred,name,query,llm_model,embedding_model,llm_before_rag
0,0,179,adult,Find me a dataset about Annual crop yield,qwen2:1.5b,BAAI/bge-large-en-v1.5,True
1,1,509,places,Find me a dataset about Annual crop yield,qwen2:1.5b,BAAI/bge-large-en-v1.5,True
2,2,4535,Census-Income,Find me a dataset about Annual crop yield,qwen2:1.5b,BAAI/bge-large-en-v1.5,True
3,3,42972,chronic-kidney-disease,Find me a dataset about Annual crop yield,qwen2:1.5b,BAAI/bge-large-en-v1.5,True
4,4,43304,US-Accidents-(4.2-million-records),Find me a dataset about Annual crop yield,qwen2:1.5b,BAAI/bge-large-en-v1.5,True


In [12]:
len(exp["y_pred"].unique())

5703

In [13]:
exp["y_true"] = exp["query"].map(query_key_dict)

In [14]:
exp["y_true"] = exp["y_true"].str.split(",")

In [25]:
exp.head(2)

,Unnamed: 0,y_pred,name,query,llm_model,embedding_model,llm_before_rag,y_true
0,0,179,adult,Find me a dataset about Annual crop yield,qwen2:1.5b,BAAI/bge-large-en-v1.5,True,[43760]
1,1,509,places,Find me a dataset about Annual crop yield,qwen2:1.5b,BAAI/bge-large-en-v1.5,True,[43760]


In [17]:
from irmetrics.topk import rr
import numpy as np

In [21]:
exp["y_true"].iloc[0]

['43760']

In [22]:
exp["y_pred"].iloc[0]

179

In [20]:
rr(exp["y_true"].iloc[0], exp["y_pred"].iloc[0])

0.0

In [ ]:
# Map true labels using the query dictionary
exp["y_true"] = exp["query"].map(query_dict)

# Calculate the number of correct and wrong predictions
exp["correct"] = exp.apply(lambda row: row["y_pred"] in row["y_true"], axis=1)
correct = exp["correct"].sum()
wrong = len(exp) - correct

# Store results in the dictionary
results_dict[folder_name] = {"correct": correct, "wrong": wrong}

In [50]:

# %%
results_dict = create_results_dict(csv_files, df_queries)


100%|██████████| 1/1 [00:01<00:00,  1.46s/it]


In [53]:

# %%
pd.DataFrame.from_dict(results_dict, orient="index")

# # %%
# pd.DataFrame.from_dict(results_dict, orient="index").to_csv(
#     "../../data/experiments/results.csv"
# )

,correct,wrong
BAAI_bge-large-en-v1.5_qwen2_1.5b,1,373174
